In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
print(os.getenv("PINECONE_API_KEY"))

32d0c4bc-58c9-4d56-a468-433c6a6777e4


In [2]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag",
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [3]:
import json
data = json.load(open("reviews.json"))
data["reviews"]

[{'professor': 'Dr. Emily Johnson',
  'subject': 'Physics',
  'stars': 4,
  'review': 'Dr. Johnson explains complex concepts clearly. Her enthusiasm for physics is contagious!'},
 {'professor': 'Prof. Michael Chen',
  'subject': 'Computer Science',
  'stars': 5,
  'review': "Brilliant instructor. Prof. Chen's projects are challenging but incredibly rewarding."},
 {'professor': 'Dr. Sarah Martinez',
  'subject': 'Biology',
  'stars': 3,
  'review': 'Knowledgeable, but lectures can be dry. More interactive sessions would be great.'},
 {'professor': 'Prof. David Taylor',
  'subject': 'History',
  'stars': 5,
  'review': 'Prof. Taylor brings history to life. His storytelling approach makes every class engaging.'},
 {'professor': 'Dr. Rachel Wong',
  'subject': 'Psychology',
  'stars': 4,
  'review': "Dr. Wong's real-world examples make psychological concepts easy to grasp and remember."},
 {'professor': 'Prof. James Anderson',
  'subject': 'Mathematics',
  'stars': 2,
  'review': 'Struggle

In [4]:
processed_data=[]
client = OpenAI()

for review in data["reviews"]:
    response = client.embeddings.create(
        input = review["review"],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })



In [5]:

processed_data[0]

{'values': [0.0060535446,
  -0.0073241233,
  -0.03177113,
  0.043186385,
  0.023322776,
  0.005338428,
  0.004769661,
  0.04507562,
  -0.004716443,
  -0.0018859119,
  0.021513365,
  0.008302004,
  -0.028285353,
  0.0028371832,
  0.02573089,
  0.033740193,
  -0.036693793,
  -0.007636779,
  0.03727919,
  0.047709912,
  0.021899195,
  -0.023016773,
  0.032941926,
  0.003021783,
  -0.041855935,
  -0.041563235,
  0.039674,
  0.024333918,
  0.031877566,
  -0.020063175,
  0.08254108,
  -0.00055130495,
  -0.01314484,
  -0.027833,
  -0.022191895,
  0.043745175,
  -0.014435375,
  0.0040711747,
  0.0024979187,
  0.0047596823,
  -0.017349059,
  -0.018759334,
  -0.03892895,
  0.015566257,
  0.030653551,
  0.002890401,
  -0.029216668,
  -0.013224666,
  0.044437006,
  0.037944414,
  0.0042141983,
  -0.017694976,
  0.011009468,
  -0.0048162267,
  -0.028045872,
  0.03195739,
  0.03932808,
  0.02730082,
  -0.00087227585,
  -0.041137494,
  0.024639921,
  -0.014980859,
  -0.028258743,
  0.015513039,
  -0.

In [6]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [7]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}